In [1]:
# # stdlib
# import os

# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
# os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"

In [2]:
# stdlib
from os import environ as env

# syft absolute
import syft as sy

In [3]:
# when in k8s these are the default values
ROOT_EMAIL = "admin@bigquery.org"
ROOT_PASSWORD = "bqpw"

In [4]:
# in case we are not in k8s we set them here for orchestra to use
env["DEFAULT_ROOT_EMAIL"] = ROOT_EMAIL
env["DEFAULT_ROOT_PASSWORD"] = ROOT_PASSWORD

In [5]:
# third party
# run email server
from helpers import EmailServer
from helpers import SMTPTestServer

email_server = EmailServer()
email_server.reset_emails()
smtp_server = SMTPTestServer(email_server)
smtp_server.start()

> Starting SMTPTestServer on: localhost:9025


In [6]:
server = sy.orchestra.launch(
    name="bigquery-high",
    dev_mode=True,
    server_side_type="high",
    reset=True,
    port="8080",
    n_consumers=1,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

Autoreload enabled
Starting bigquery-high server on 0.0.0.0:8080


INFO:     Will watch for changes in these directories: ['/Users/madhavajay/dev/PySyft/packages/syft/src/syft']
INFO:     Uvicorn running on http://0.0.0.0:8080 (Press CTRL+C to quit)
INFO:     Started reloader process [12250] using WatchFiles


Found `reset=True` in the launch configuration. Resetting the server...


INFO:     Started server process [12265]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


WARN: private key is based on server name: bigquery-high in dev_mode. Don't run this in production.
Document Store's SQLite DB path: /var/folders/6_/7xx0tpq16h9cn40mq4w5gjk80000gn/T/syft/fbdf5a287e58454cbbd3fac4ad744d37/db/fbdf5a287e58454cbbd3fac4ad744d37.sqlite
Action Store's SQLite DB path: /var/folders/6_/7xx0tpq16h9cn40mq4w5gjk80000gn/T/syft/fbdf5a287e58454cbbd3fac4ad744d37/db/fbdf5a287e58454cbbd3fac4ad744d37.sqlite
INFO:     127.0.0.1:63395 - "GET /api/v2/metadata HTTP/1.1" 200 OK
 Done.


SyftInfo: You have launched a development server at http://0.0.0.0:8080.It is intended only for local use.

In [7]:
root_client = sy.login(
    url="http://localhost:8080", email=ROOT_EMAIL, password=ROOT_PASSWORD
)

INFO:     127.0.0.1:63400 - "GET /api/v2/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:63400 - "POST /api/v2/login HTTP/1.1" 200 OK
INFO:     127.0.0.1:63400 - "GET /api/v2/api?verify_key=1e5ec44a179d21e0a0721ea7a670b9875d3946c8604f4623903fcafafe03c8e7&communication_protocol=dev HTTP/1.1" 200 OK
INFO:     127.0.0.1:63402 - "POST /api/v2/api_call HTTP/1.1" 200 OK
Logged into <bigquery-high: High side Datasite> as <admin@bigquery.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].account.set_password([new_password])`.

# Create new admin client

In [8]:
ADMIN_EMAIL, ADMIN_PW = "admin2@bigquery.org", "bqpw2"

In [9]:
# create
root_client.register(
    name="second admin", email=ADMIN_EMAIL, password=ADMIN_PW, password_verify=ADMIN_PW
)
# update role
new_user_id = root_client.users.search(email=ADMIN_EMAIL)[0].id
root_client.users.update(uid=new_user_id, role="admin")

INFO:     127.0.0.1:63400 - "POST /api/v2/register HTTP/1.1" 200 OK
INFO:     127.0.0.1:63404 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:63406 - "POST /api/v2/api_call HTTP/1.1" 200 OK


```python
class UserView:
  id: str = 029260e6abfe4c3dad3f35db8119bd56
  name: str = "second admin"
  email: str = "admin2@bigquery.org"
  institution: str = None
  website: str = None
  role: str = ServiceRole.ADMIN
  notifications_enabled: str = {: True, : False, : False, : False}

```

# We cannot delete the root client

In [10]:
high_client = sy.login(
    url="http://localhost:8080", email=ADMIN_EMAIL, password=ADMIN_PW
)

INFO:     127.0.0.1:63408 - "GET /api/v2/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:63408 - "POST /api/v2/login HTTP/1.1" 200 OK
INFO:     127.0.0.1:63408 - "GET /api/v2/api?verify_key=e1a3b4c8ca44fc54d1a503cf21439ed86ba9c4c244b0d07de036b4e2d8f9bf25&communication_protocol=dev HTTP/1.1" 200 OK
INFO:     127.0.0.1:63410 - "POST /api/v2/api_call HTTP/1.1" 200 OK
Logged into <bigquery-high: High side Datasite> as <admin2@bigquery.org>


In [11]:
root_admin_id = root_client.users.search(email=ROOT_EMAIL)[0].id

INFO:     127.0.0.1:63413 - "POST /api/v2/api_call HTTP/1.1" 200 OK


In [12]:
with sy.raises(sy.SyftException):
    high_client.users.delete(root_admin_id)

INFO:     127.0.0.1:63415 - "POST /api/v2/api_call HTTP/1.1" 200 OK


# Create ephemeral admin and delete it

In [13]:
# register 2nd new admin (to delete)
user_email, user_pw = "admin3@bigquery.org", "bqpw3"
## create
root_client.register(
    name="x", email=user_email, password=user_pw, password_verify=user_pw
)
## update role
new_user_id2 = root_client.users.search(email=user_email)[0].id
root_client.users.update(uid=new_user_id, role="admin")

INFO:     127.0.0.1:63400 - "POST /api/v2/register HTTP/1.1" 200 OK
INFO:     127.0.0.1:63417 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:63419 - "POST /api/v2/api_call HTTP/1.1" 200 OK


```python
class UserView:
  id: str = 029260e6abfe4c3dad3f35db8119bd56
  name: str = "second admin"
  email: str = "admin2@bigquery.org"
  institution: str = None
  website: str = None
  role: str = ServiceRole.ADMIN
  notifications_enabled: str = {: True, : False, : False, : False}

```

In [14]:
root_client.users.delete(new_user_id2)

INFO:     127.0.0.1:63421 - "POST /api/v2/api_call HTTP/1.1" 200 OK


<UID: c17dd7791c9e481d9d4c99175a5f5fd0>

In [15]:
smtp_server.stop()

> Stopping SMTPTestServer


In [16]:
server.land()

Stopping bigquery-high


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [12265]
INFO:     Stopping reloader process [12250]
